In [9]:
import time

# First exercise: count the number of lines in Python for each file

In [12]:
timeB = time.time()
countB = 0
for line in open('Data/bookings.csv'): countB += 1
print('count number of Booking Data took: '+str(time.time()-timeB))

Booking Data took: 178.203999996


In [13]:
print(countB)

10000011


In [10]:
timeS = time.time()
countS = 0
for line in open('Data/searches.csv'): countS += 1
print('count number of Searches Data took: '+str(time.time()-timeS))

Searches Data took: 160.325999975


In [11]:
print(countS)

20390199


# Second exercise: top 10 arrival airports in the world in 2013

Arrival airport is the column arr_port. It is the IATA code for the airport

To get the total number of passengers for an airport, you can sum the column pax, grouping by arr_port. Note that there is negative pax. That corresponds to cancelations. So to get the total number of passengers that have actually booked, you should sum including the negatives (that will remove the cancelled bookings).

Print the top 10 arrival airports in the standard output, including the number of passengers.

Bonus point: Get the name of the city or airport corresponding to that airport (programatically, we suggest to have a look at GeoBases in Github)

Bonus point: Solve this problem using pandas (instead of any other approach)

In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine # database connection
from GeoBases import GeoBase

In [ ]:
#remove space blanks from string values
def strip(text):
    try:
        return str(text).strip() #avoid mixed types
    except AttributeError:
        return ''

In [6]:
disk_engine = create_engine('sqlite:///Data/database.db') # Initializes database with filename database.db in current directory

In [8]:
#start = dt.datetime.now()
chunksize = 200000
j = 1
index_start = 1

for df in pd.read_csv('Data/bookings.csv', chunksize=chunksize, iterator=True, encoding='utf-8', sep ='^'):
    
    df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) # Remove spaces from columns
    df.index += index_start
    
    for col in df.columns:
        if df.dtypes[col].name == 'object':
            df[col] = df[col].map(lambda row:strip(row))
        else:
            df[col] = df[col].map(lambda row:make_int(row))
                      
    
    print(str(j)+'. saving in sql')
    
    df.to_sql('bookings', disk_engine, if_exists='append')
    index_start = df.index[-1] + 1
    j = j + 1

saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql
saving in sql


C:\Users\miou\Anaconda2\lib\site-packages\ipykernel\ipkernel.py:175: DtypeWarning: Columns (7,8,27) have mixed types. Specify dtype option on import or set low_memory=False.
  shell.run_cell(code, store_history=store_history, silent=silent)


In [31]:
df = pd.read_sql_query('SELECT arr_port, sum(pax) as `sum`'
                       'FROM bookings '
                       'GROUP BY arr_port ', disk_engine)

In [26]:
topArr = df.sort_values('sum',ascending = False)[:10]

In [27]:
geo_a = GeoBase(data='airports', verbose=False)
topArr['arr_city'] = topArr['arr_port'].map(lambda value: geo_a.get(value, 'name'))

In [28]:
topArr

,arr_port,sum,arr_city
1088,LHR,86836,London Heathrow Airport
1190,MCO,69199,Orlando International Airport
1050,LAX,69072,Los Angeles International Airport
1047,LAS,68117,McCarran International Airport
886,JFK,64872,John F Kennedy International Airport
315,CDG,63114,Paris - Charles-de-Gaulle
216,BKK,58180,Suvarnabhumi
1228,MIA,56859,Miami International Airport
1719,SFO,56679,San Francisco International Airport
517,DXB,54443,Dubai International Airport
